***Project Scenario:***

You have been hired as a data engineer by research organization. Your boss has asked you to create a code that can be used to compile the list of the top 10 largest banks in the world ranked by market capitalization in billion USD. Further, the data needs to be transformed and stored in GBP, EUR and INR as well, in accordance with the exchange rate information that has been made available to you as a CSV file. The processed information table is to be saved locally in a CSV format and as a database table.

Your job is to create an automated system to generate this information so that the same can be executed in every financial quarter to prepare the report.

**Task 1:**

Write a function log_progress() to log the progress of the code at different stages in a file code_log.txt. Use the list of log points provided to create log entries as every stage of the code.

**Task 2:**

Extract the tabular information from the given URL under the heading 'By market capitalization' and save it to a dataframe.
a. Inspect the webpage and identify the position and pattern of the tabular information in the HTML code
b. Write the code for a function extract() to perform the required data extraction.
c. Execute a function call to extract() to verify the output.

**Task 3:**

Transform the dataframe by adding columns for Market Capitalization in GBP, EUR and INR, rounded to 2 decimal places, based on the exchange rate information shared as a CSV file.
a. Write the code for a function transform() to perform the said task.
b. Execute a function call to transform() and verify the output.

**Task 4:**

Load the transformed dataframe to an output CSV file. Write a function load_to_csv(), execute a function call and verify the output.

**Task 5:**

Load the transformed dataframe to an SQL database server as a table. Write a function load_to_db(), execute a function call and verify the output.

**Task 6:**

Run queries on the database table. Write a function load_to_db(), execute a given set of queries and verify the output.

**Task 7:**

Verify that the log entries have been completed at all stages by checking the contents of the file code_log.txt.

**Preliminaries:**

Installing libraries:

python3.11 -m pip install pandas

python3.11 -m pip install bs4

python3.11 -m pip install numpy

Webpage: https://web.archive.org/web/20230908091635%20/https://en.wikipedia.org/wiki/List_of_largest_banks


In [41]:
# Importing the required libraries


from bs4 import BeautifulSoup
import requests
import pandas as pd
import sqlite3
from datetime import datetime
import numpy as np
import pyodbc

In [42]:

url = 'https://web.archive.org/web/20230908091635/https://en.wikipedia.org/wiki/List_of_largest_banks'

table_attribs = ["Name", "MC_USD_Billion"]
output_path = 'C:/Users/jyoth/Downloads/Python_ETL_Project/Largest_banks_data.csv'
server_name = 'JYOTHI\SQLEXPRESS'
db_name = 'Banks.db'
table_name = 'Largest_banks'
csv_path= 'C:/Users/jyoth/Downloads/Python_ETL_Project/exchange_rate.csv'

**Task 1: Logging function**

The function to log the progress of the code, log_progress(). The function accepts the message to be logged and enters it to a text file code_log.txt

Each log entry must happen in the next line in the text file.

Correct log entries must be associated with each of the executed function calls. Use the following table to note the logging message at the end of each function call that follows.    


| **Task**                        | **Log message on completion**                                      |
|---------------------------------|--------------------------------------------------------------------|
| `Declaring known values`        | `Preliminaries complete. Initiating ETL process`                  |
| `Call extract() function`       | `Data extraction complete. Initiating Transformation process`      |
| `Call transform() function`     | `Data transformation complete. Initiating Loading process`        |
| `Call load_to_csv()`            | `Data saved to CSV file`                                          |
| `Initiate SQLite3 connection`   | `SQL Connection initiated`                                        |
| `Call load_to_db()`             | `Data loaded to Database as a table, Executing queries`           |
| `Call run_query()`              | `Process Complete`                                                |
| `Close SQLite3 connection`      | `Server Connection closed`                                        |



In [43]:
# Function to log progress
def log_progress(message):
    '''Logs a given message with a timestamp'''
    timestamp_format = '%Y-%m-%d %H:%M:%S' 
    now = datetime.now()  
    timestamp = now.strftime(timestamp_format) 
    with open("C:/Users/jyoth/Downloads/Python_ETL_Project/code_log.txt", "a") as f: 
        f.write(timestamp + ' : ' + message + '\n')
        
log_progress('Preliminaries complete. Initiating ETL process')



**Task 2 : Extraction of data**

Analyze the webpage on the given URL:


URL= https://web.archive.org/web/20230908091635/https://en.wikipedia.org/wiki/List_of_largest_banks

Identify the position of the required table under the heading By market capitalization. Write the function extract() to retrieve the information of the table to a Pandas data frame.

Note: Remember to remove the last character from the Market Cap column contents, like, '\n', and typecast the value to float format.

Write a function call for extract() and print the returning data frame.

Make the relevant log entry.

In [44]:
# Extract function
def extract(url, table_attribs):
    '''Extracts the first table from Wikipedia'''
    page = requests.get(url).text
    data = BeautifulSoup(page, 'html.parser')
    df = pd.DataFrame(columns=table_attribs)
    
    tables = data.find_all('tbody')  # Locate all table bodies
    if len(tables) == 0:
        log_progress("Error: No tables found on the page.")
        return None
    
    rows = tables[0].find_all('tr')  # First table (index 0)
    for row in rows:
        col = row.find_all('td')
        if len(col) >= 3:  # Ensure we have enough columns
            name = col[1].get_text(strip=True)  # Extract bank name from the correct column
            mc_usd_billion = col[2].get_text(strip=True)  # Extract market cap from the correct column
            df = pd.concat([df, pd.DataFrame([[name, mc_usd_billion]], columns=table_attribs)], ignore_index=True)
    
    return df

# Extract data and print results
df = extract(url, table_attribs)
print(df)
log_progress('Data extraction complete. Initiating Transformation process')


                                      Name MC_USD_Billion
0                           JPMorgan Chase         432.92
1                          Bank of America         231.52
2  Industrial and Commercial Bank of China         194.56
3               Agricultural Bank of China         160.68
4                                HDFC Bank         157.91
5                              Wells Fargo         155.87
6                        HSBC Holdings PLC         148.90
7                           Morgan Stanley         140.83
8                  China Construction Bank         139.82
9                            Bank of China         136.81


**Task 3 : Transformation of data**

The Transform function needs to perform the following tasks:

Read the exchange rate CSV file and convert the contents to a dictionary so that the contents of the first columns are the keys to the dictionary and the contents of the second column are the corresponding values.

Add 3 different columns to the dataframe,  MC_GBP_Billion, MC_EUR_Billion and MC_INR_Billion, each containing the content of MC_USD_Billion scaled by the corresponding exchange rate factor. Round the resulting data to 2 decimal places.

Write the function call for transform() and print the contents of the returning data frame. Comment out all previous print statements.

Make the relevant log entry and execute the code.

***Print the contents of df['MC_EUR_Billion'][4], which is the market capitalization of the 5th largest bank in billion EUR.***

In [45]:
def transform(df, csv_path):
    ''' This function accesses the CSV file for exchange rate
    information, and adds three columns to the data frame, each
    containing the transformed version of Market Cap column to
    respective currencies'''
    
    # Load exchange rate CSV file
    exchange_df = pd.read_csv(csv_path)
    
    # Convert exchange rates DataFrame to dictionary
    exchange_rate = exchange_df.set_index('Currency')['Rate'].to_dict()
    new_columns = ["MC_GBP_Billion", "MC_EUR_Billion", "MC_INR_Billion"]

# Add them to the DataFrame with default values as None
    for col in new_columns:
        df[col] = None
    df['MC_USD_Billion']= df['MC_USD_Billion'].astype(float)
    df['MC_GBP_Billion'] = [np.round(x*exchange_rate['GBP'],2) for x in df['MC_USD_Billion']]
    df['MC_EUR_Billion'] = [np.round(x*exchange_rate['EUR'],2) for x in df['MC_USD_Billion']]
    df['MC_INR_Billion'] = [np.round(x*exchange_rate['INR'],2) for x in df['MC_USD_Billion']]
    return df

df=transform(df, csv_path)

print(df)

log_progress('Data transformation complete. Initiating Loading process')

print(df.loc[4, 'MC_EUR_Billion'])

                                      Name  MC_USD_Billion  MC_GBP_Billion  \
0                           JPMorgan Chase          432.92          346.34   
1                          Bank of America          231.52          185.22   
2  Industrial and Commercial Bank of China          194.56          155.65   
3               Agricultural Bank of China          160.68          128.54   
4                                HDFC Bank          157.91          126.33   
5                              Wells Fargo          155.87          124.70   
6                        HSBC Holdings PLC          148.90          119.12   
7                           Morgan Stanley          140.83          112.66   
8                  China Construction Bank          139.82          111.86   
9                            Bank of China          136.81          109.45   

   MC_EUR_Billion  MC_INR_Billion  
0          402.62        35910.71  
1          215.31        19204.58  
2          180.94        16138.75

**Task 4: Loading to CSV**

Below is the function to load the transformed data frame to a CSV file, like load_to_csv(), in the output path 

Make the relevant log entry.

In [46]:
def load_to_csv(df, output_path):
    ''' This function saves the final data frame as a CSV file in
    the provided path. Function returns nothing.'''
    df.to_csv(output_path)

load_to_csv(df, output_path)
log_progress('Data saved to CSV file')

**Task 5: Loading to Database**

Write the function to load the transformed data frame to an SQL database, like, load_to_db(). Use the database and table names as mentioned in the project scenario.

Before calling this function, initiate the connection to the SQL database server with the name Banks.db. Pass this connection object, along with the required table name Largest_banks and the transformed data frame, to the load_to_db() function in the function call.

Make the relevant log entry.

Upon successful function call, you will have loaded the contents of the table with the required data.

In [47]:
def load_to_db(df, server_name, db_name, table_name):
    '''This function saves the final dataframe to a SQL Server table.'''
    
    # Establish connection to local SQL Server
    conn = pyodbc.connect(f'DRIVER=SQL Server;SERVER={server_name};DATABASE={db_name};Trusted_Connection=yes;')
    cursor = conn.cursor()
    
    # Drop table if it exists
    drop_table_query = f"IF OBJECT_ID('{table_name}', 'U') IS NOT NULL DROP TABLE {table_name};"
    cursor.execute(drop_table_query)
    conn.commit()

    
    # Generate column definitions with correct data types
    sql_types = {
        'int64': 'INT',
        'float64': 'FLOAT',
        'object': 'NVARCHAR(255)'
    }
    
    column_defs = ', '.join([f'[{col}] {sql_types[str(df[col].dtype)]}' for col in df.columns])
    
    # Create table if not exists
    create_table_query = f"""
    IF NOT EXISTS (SELECT * FROM sys.tables WHERE name = '{table_name}')
    CREATE TABLE {table_name} ({column_defs})
    """
    cursor.execute(create_table_query)
    conn.commit()

    # Insert data
    placeholders = ', '.join(['?' for _ in df.columns])
    insert_query = f"INSERT INTO {table_name} VALUES ({placeholders})"

    for _, row in df.iterrows():
        cursor.execute(insert_query, tuple(row))

    conn.commit()
    cursor.close()
    conn.close()
    print(f"Data loaded into {table_name} in SQL Server.")



log_progress('SQL Connection initiated.')

# Load DataFrame into SQL Server
load_to_db(df, server_name, db_name, table_name)

log_progress('Data loaded to Database as table. Running the query.')

Data loaded into Largest_banks in SQL Server.


**Task 6: Function to Run queries on Database**

Below is the function run_queries() that accepts the query statement, and the SQL Server Connection object, and generates the output of the query. The query statement should be printed along with the query output.

Execute 3 function calls using the queries as mentioned below.

1. Print the contents of the entire table

Query statement:   SELECT * FROM Largest_banks

2. Print the average market capitalization of all the banks in Billion USD.

Query statement:   SELECT AVG(MC_GBP_Billion) FROM Largest_banks

3. Print only the names of the top 5 banks

Query statement:   SELECT TOP 5 Name FROM Largest_banks

Make the relevant log entry.

In [48]:
conn = pyodbc.connect(f'DRIVER=SQL Server;SERVER={server_name};DATABASE={db_name};Trusted_Connection=yes;')
cursor = conn.cursor()

def run_query(query_statement, conn):
    print(query_statement)
    query_output = pd.read_sql(query_statement, conn)
    print(query_output)

query_statement = f"SELECT * FROM Largest_banks"
run_query(query_statement, conn)

query_statement = f"SELECT AVG(MC_GBP_Billion) FROM Largest_banks"
run_query(query_statement, conn)

query_statement = f"SELECT TOP 5 Name FROM Largest_banks"
run_query(query_statement, conn)

log_progress('Process Complete.')

conn.close()

log_progress('Server Connection closed')

SELECT * FROM Largest_banks
                                      Name  MC_USD_Billion  MC_GBP_Billion  \
0                           JPMorgan Chase          432.92          346.34   
1                          Bank of America          231.52          185.22   
2  Industrial and Commercial Bank of China          194.56          155.65   
3               Agricultural Bank of China          160.68          128.54   
4                                HDFC Bank          157.91          126.33   
5                              Wells Fargo          155.87          124.70   
6                        HSBC Holdings PLC          148.90          119.12   
7                           Morgan Stanley          140.83          112.66   
8                  China Construction Bank          139.82          111.86   
9                            Bank of China          136.81          109.45   

   MC_EUR_Billion  MC_INR_Billion  
0          402.62        35910.71  
1          215.31        19204.58  
2    

C:\Users\jyoth\AppData\Local\Temp\ipykernel_15612\2591526257.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  query_output = pd.read_sql(query_statement, conn)
C:\Users\jyoth\AppData\Local\Temp\ipykernel_15612\2591526257.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  query_output = pd.read_sql(query_statement, conn)
C:\Users\jyoth\AppData\Local\Temp\ipykernel_15612\2591526257.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  query_output = pd.read_sql(query_statement, conn)


**Task 7: Verify log entries**

After updating all the log_progress() function calls, you have to run the code for a final execution. However, you will first have to remove the code_log.txt file, that would have been created and updated throughout the multiple executions of the code 
and run the final execution. Upon successful completion of execution, open the code_log.txt file. Now you should see all the relevant entries made in the text file in relation to the stages of code execution.